# Retrieval-Augmented Shopping Assistant - EDA

This notebook explores the ABO dataset for initial insights.

In [6]:
import pandas as pd
import json
import zipfile

In [ ]:
# Path to your zip file
zip_path = '/content/sample_data/listings_0.json'

# Open the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # List all files inside the zip
    print("Files in the zip:")
    print(zip_ref.namelist())
    metadata_files = [f for f in zip_ref.namelist() if f.endswith('.json')]
    for file_name in metadata_files:



Files in the zip:
['LICENSE-CC-BY-4.0.txt', 'listings/README.md', 'listings/metadata/listings_0.json', 'listings/metadata/listings_1.json', 'listings/metadata/listings_2.json', 'listings/metadata/listings_3.json', 'listings/metadata/listings_4.json', 'listings/metadata/listings_5.json', 'listings/metadata/listings_6.json', 'listings/metadata/listings_7.json', 'listings/metadata/listings_8.json', 'listings/metadata/listings_9.json', 'listings/metadata/listings_a.json', 'listings/metadata/listings_b.json', 'listings/metadata/listings_c.json', 'listings/metadata/listings_d.json', 'listings/metadata/listings_e.json', 'listings/metadata/listings_f.json']
Reading file: listings/metadata/listings_0.json
Reading file: listings/metadata/listings_1.json
Reading file: listings/metadata/listings_2.json
Reading file: listings/metadata/listings_3.json
Reading file: listings/metadata/listings_4.json
Reading file: listings/metadata/listings_5.json
Reading file: listings/metadata/listings_6.json
Readin

In [7]:
# Preview
import json
import pandas as pd
valid_data=[]

# Load your JSON (string or from a file)
with open('/content/sample_data/listings_1.json', 'r', encoding='utf-8') as f:
   for line in f:
    try:
      data = json.loads(line)
      valid_data.append(data)
    except json.JSONDecodeError as e:
      print(f"Error decoding JSON: {e}")

Error decoding JSON: Unterminated string starting at: line 1 column 9647 (char 9646)


In [8]:

def remove_language_tag(obj):
    """Recursively remove 'language_tag' keys from nested dicts and lists."""
    if isinstance(obj, dict):
        return {k: remove_language_tag(v) for k, v in obj.items() if k != "language_tag"}
    elif isinstance(obj, list):
        return [remove_language_tag(item) for item in obj]
    else:
        return obj


In [9]:
cleaned_data = []
# with open("/content/sample_data/listings_1.json", "r", encoding="utf-8") as f:
for line in valid_data:
    # data = json.loads(line)
    cleaned = remove_language_tag(line)
    cleaned_data.append(cleaned)

In [10]:
def flatten_record(record):
    flat = {}
    for key, value in record.items():
        if isinstance(value, list) and all(isinstance(v, dict) and "value" in v for v in value):
            # Join multiple values with comma
            flat[key] = ", ".join(str(v["value"]) for v in value)
        else:
            flat[key] = value
    return flat

In [21]:
# Apply to all records
flattened = [flatten_record(rec) for rec in cleaned_data]

# Convert to DataFrame
df = pd.DataFrame(flattened)
df.to_csv('sample_data/shopAssist_Ori.csv', index=False)
df.iloc[:, 25:]

,finish_type,product_description,pattern
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
2327,NaN,NaN,Solid
2328,NaN,NaN,NaN
2329,NaN,NaN,NaN
2330,NaN,NaN,NaN


In [12]:
df.drop(columns=['item_id','model_number','main_image_id','other_image_id', 'domain_name', 'model_year','spin_id','3dmodel_id','finish_type','pattern'],inplace=True)


In [13]:
df.describe()

,brand,bullet_point,color,item_name,item_weight,model_name,product_type,item_keywords,country,marketplace,node,item_shape,material,style,item_dimensions,color_code,fabric_type,product_description
count,2331,2079,1840,2332,1634,1298,2332,1999,2332,2332,2225,89,840,721,685,289,121,71
unique,134,1521,583,2328,425,702,221,1480,17,5,899,38,251,522,629,155,87,64
top,Amazon Brand - Solimo,3D Printed Hard Back Case Mobile Cover for App...,Others,Amazon Brand - Symbol Men's Formal Shoes,50,Samsung Galaxy M31,CELLULAR_PHONE_CASE,"mobile cover, back cover, mobile case, phone c...",IN,Amazon,"[{'node_id': 12710103031, 'node_name': '/Categ...",Rectangular,Plastic,pattern,{'height': {'normalized_value': {'unit': 'inch...,[#242124],100% Polyester,An Amazon Brand
freq,1049,10,512,2,992,16,998,26,1182,2243,671,25,170,25,10,33,7,5


In [31]:
df.fillna(0,inplace=True)

In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 62.8 MB/s eta 0:00:00


In [3]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [14]:
from re import S
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

#store df into csv file
df.to_csv('sample_data/shopAssist.csv', index=False)
#read file from sample_data
df_file = pd.read_csv('sample_data/shopAssist.csv')
#combined all the columns into one
df['combined'] = df_file.apply(lambda x: ' '.join(x.astype(str)), axis=1)
print (df['combined'][0])

#Generate embeddings



Amazon Brand - Solimo 3D Printed Hard Back Case Mobile Cover for Coolpad Cool1 dual, Easy to put & take off with perfect cutouts for volume buttons, audio & charging ports., Stylish design and appearance, express your unique personality., Extreme precision design allows easy access to all buttons and ports while featuring raised bezel to life screen and camera off flat surface., Slim Hard Back Cover, No Warranty Others Amazon Brand - Solimo Designer Leafs 3D Printed Hard Back Case Mobile Cover for Coolpad Cool1 Dual 50.0 Coolpad Cool1 dual CELLULAR_PHONE_CASE mobile cover, back cover, mobile case, phone case, mobile panel, phone panel, hard case, 3D printed mobile cover, mobile cover, back cover, mobile case, phone case, mobile panel, phone panel, hard case, 3D printed mobile cover, mobile cover, back cover, mobile case, phone case, mobile panel, phone panel, hard case, 3D printed mobile cover, mobile cover, back cover, mobile case, phone case, mobile panel, phone panel, hard case, 3D 

In [15]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(df['combined'], convert_to_tensor=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [79]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

In [80]:
def search(query, top_k=2):
    query_vector = model.encode([query])
    distances, indices = index.search(query_vector, top_k)
    df = pd.read_csv('sample_data/shopAssist_Ori.csv')
    return df.iloc[indices[0]]

In [109]:
result_df = search("Sterling Silver")
images_df = pd.read_csv('sample_data/images.csv')
image_id = result_df['main_image_id'].tolist()
# image_mapping = images_df.set_index('image_id')['path'].to_dict()
image_mapping = images_df[images_df['image_id'].isin(image_id)]['path'].tolist()

#Map each image_id to its path and prepend "sample_data/"
# result_df['main_image_id'] = result_df['main_image_id'].map(lambda x: f"sample_data/{image_mapping.get(x)}")
# result_df['main_image_id']

None


,main_image_id
2112,sample_data/12/12d03bf2.jpg
511,sample_data/51/519a3e44.jpg


In [114]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
for url in image_mapping.keys():
  print(url)
  # image_path = "sample_data/"+url.get
  # img = mpimg.imread(image_path)
  # plt.imshow(img)
  # plt.axis('off')
  # plt.show()

Streaming output truncated to the last 5000 lines.
A1hFW0YsvAL
A1hFXE9J9OL
A1hFf5kUGjL
A1hG2nLN-PL
A1hGFXdq5wL
A1hGH0rFugL
A1hGQ97nBYL
A1hHTuJmLCL
A1hHZxBV4JL
A1hHvqSilyL
A1hI5NXkZPL
A1hIsvTytBL
A1hJ3hTgQqL
A1hJCgmohrL
A1hJEBuv0-L
A1hKe5mVsbL
A1hKrCINszL
A1hLC3VjKeL
A1hLTzxinNL
A1hNm4T+muL
A1hO403H1nL
A1hOkMf3WuL
A1hOpm+-dvL
A1hOrvWA2RL
A1hPIYRuUML
A1hPR3-gTzL
A1hPYm2qoJL
A1hQXtmmukL
A1hQlKrvzCL
A1hQuNDnTdL
A1hR3cg2VDL
A1hR8CUrUvL
A1hS4nukPeL
A1hScAAZzbL
A1hSd+4h9PL
A1hSnSDcQKL
A1hTIWjfo0L
A1hTcSijr4L
A1hUHXRce8L
A1hUHxOTmKL
A1hUoWmCRzL
A1hVYcxq3kL
A1hVbmpKOiL
A1hVtvDjC7L
A1hWFLGpNdL
A1hXC5DGeVL
A1hYG7PXGUL
A1hYIyDDklL
A1hYqbrFrUL
A1hYv4CdNtL
A1hZbfyNKdL
A1hZlbwWzUL
A1hZwO34p-L
A1ha1NnpOwL
A1ha9JfY4aL
A1haCdQGkHL
A1haEnwAI0L
A1haQJcqbpL
A1haY58FPdL
A1hb+wxjphL
A1hbYzmsIwL
A1hbhu0BCJL
A1hc22CqZ4L
A1hc3AJ2LZL
A1hc3YM6OYL
A1hc9T34CXL
A1hcImPYVbL
A1hcZiOkLwL
A1hcjNMMAcL
A1hdEN3RGhL
A1hdcN4V4LL
A1heGOUCJGL
A1heV6uC1zL
A1hee6Nd6DL
A1hf-JU9ufL
A1hf0IW3vML
A1hfJKI7xgL
A1hfW8RQ0bL
A1hg9LUfSJL
A